In [ ]:
import numpy as np

###########
# constants
###########

n =  # number of stacs
m_stac =  # mass of individual stac

R_z_dot = # tbd, this is our stack velocity in the z direction
R_z = # array of vertical positions of each stack

dt = 1

w_motor =  # motor ang velocity (radians per second)
l_screw =  # screw lead (meters)
r_screw =  # screw radius (meters)
v =  (w_motor * l_screw) / (2 * np.pi)  # linear velocity of screw shaft (meters per second)


MOI_cube =  #3x3 matrix for moment of inertia of cubesat without stacs
MOI_stac = #3x3 matrix giving moment of inertia of an individual stac about it's own axis
n = #number of stacs left in the cubesat
displace_data = #nx3 matrix giving the x y z locations of the stacs (will need to code up function for this in future)
m_stac = #mass of stac 

##################
# helper functions
##################

def make_J_dot(R_x, R_y, R_z_dot):
    xz = -n * m_stac * R_x * R_z_dot
    yz = -n * m_stac * R_y * R_z_dot
    return np.matrix([[0,  0,  xz],
                      [0,  0,  yz],
                      [xz, yz, 0]])

#find moment of inertia of cubesat

def inertia_of_cubesat(MOI_cube,MOI_stac,n,displace_data):
    
    MOI_mat = np.empty([3,3])
    
    for i in range(3):
        for j in range(3):
            for x in range(n):
                if i != j:
                    g = 0
                else:
                    g = displace_data[x,i] * displace_data[x,j]
            ab += g  
            
        MOI_mat[i,j] = MOI_cube[i,j] + n * MOI_stac[i,j] + m*ab
        
    return MOI_mat
            
    






In [ ]:
#driving torque required - checking if it below stepper motor 

linear_thrust = # linear thrust required to drive PCBsats
efficiency = #efficiency leadscrew

driving_torque = (l_screw * linear_thrust)/(2 * np.pi * efficiency)


